In [4]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [0]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re

In [0]:
fields = ['Event 1', 'Event 2', 'Label']

df_train = pd.read_csv('P1_training_set.csv', skipinitialspace=True, usecols=fields)
df_test = pd.read_csv('P1_testing_set.csv', skipinitialspace=True, usecols=fields)

In [0]:
from nltk.stem.snowball import SnowballStemmer
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    
    return text
  
# apply the above function to df['text']
df_train['Event 1'] = df_train['Event 1'].map(lambda x: clean_text(x))
df_train['Event 2'] = df_train['Event 2'].map(lambda x: clean_text(x))

df_test['Event 1'] = df_test['Event 1'].map(lambda x: clean_text(x))
df_test['Event 2'] = df_test['Event 2'].map(lambda x: clean_text(x))

In [0]:
### Create sequence
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)

# training data
tokenizer.fit_on_texts(df_train['Event 1'])
sequences1 = tokenizer.texts_to_sequences(df_train['Event 1'])
data1 = pad_sequences(sequences1, maxlen=100)

tokenizer.fit_on_texts(df_train['Event 2'])
sequences2 = tokenizer.texts_to_sequences(df_train['Event 2'])
data2 = pad_sequences(sequences2, maxlen=100)

data_train = np.concatenate([data1, data2], axis = 1)

# testing data
tokenizer.fit_on_texts(df_test['Event 1'])
sequences1_test = tokenizer.texts_to_sequences(df_test['Event 1'])
data1_test = pad_sequences(sequences1_test, maxlen=100)

tokenizer.fit_on_texts(df_test['Event 2'])
sequences2_test = tokenizer.texts_to_sequences(df_test['Event 2'])
data2_test = pad_sequences(sequences2_test, maxlen=100)

data_test = np.concatenate([data1_test, data2_test], axis = 1)

In [0]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [0]:
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [24]:
## Network architecture
labels = df_train['Label']

## create model
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=200, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(Flatten())
model_glove.add(Dense(3, activation='softmax'))
model_glove.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_glove.fit(data_train, labels, epochs = 100)

model_glove.evaluate(data_test, y = df_test['Label'])

Epoch 1/100
14358/14358 [==============================] - 3s 227us/step - loss: 0.9791 - acc: 0.4721
Epoch 2/100
14358/14358 [==============================] - 3s 180us/step - loss: 0.9495 - acc: 0.5180
Epoch 3/100
14358/14358 [==============================] - 3s 193us/step - loss: 0.9251 - acc: 0.5517
Epoch 4/100
14358/14358 [==============================] - 3s 195us/step - loss: 0.8980 - acc: 0.5794
Epoch 5/100
14358/14358 [==============================] - 3s 198us/step - loss: 0.8741 - acc: 0.5958
Epoch 6/100
14358/14358 [==============================] - 3s 196us/step - loss: 0.8407 - acc: 0.6165
Epoch 7/100
14358/14358 [==============================] - 3s 197us/step - loss: 0.8106 - acc: 0.6344
Epoch 8/100
14358/14358 [==============================] - 3s 197us/step - loss: 0.7828 - acc: 0.6474
Epoch 9/100
14358/14358 [==============================] - 3s 198us/step - loss: 0.7651 - acc: 0.6649
Epoch 10/100
14358/14358 [==============================] - 3s 197us/step - loss: 

[1.7686843101976824, 0.41971481733268934]

In [25]:
print(model_glove.metrics_names)

['loss', 'acc']
